In [21]:
!pip install psycopg2
import psycopg2 as ps #postgress wrapper
import pandas as pd
#!pip install PyMySQL
#import pymysql
#db = pymysql.connect('.rds.amazonaws.com', '(***)', '***') #

In [22]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [23]:
df = pd.read_csv('/content/drive/My Drive/api-youtube-main/youtube_vids_2nd_pull.csv', index_col=0) 
df.head()

,video_id,video_title,upload_date,upload_status,privacy_status,license,madeForKids
0,ToEl_tpyoM4,Uber Data Science Python Interview Question Wa...,2022-01-10,processed,public,youtube,False
1,30hS-MjpU6E,SQL Syntax Best Practices: How to Structure Yo...,2021-12-22,processed,public,youtube,False
2,Bgpp99iz0I0,Most Common Coding Mistakes on Data Science In...,2021-11-04,processed,public,youtube,False
3,GeJUvdkJKEc,Multiple Solutions to Data Scientist Interview...,2021-10-11,processed,public,youtube,False
4,GGURenNfXI0,Multiple Approaches for Solving a Twitter Data...,2021-09-13,processed,public,youtube,False


In [30]:
def connect_to_db(host_name, dbname, port, username, password):
    try:
        conn = ps.connect(host=host_name, database=dbname, user=username, password=password, port=port)
    except ps.OperationalError as e:
        raise e
    else:
        print('Connected!')
        return conn

def create_table(curr):
    create_table_command = ("""CREATE TABLE IF NOT EXISTS videos (
                    video_id VARCHAR(255) PRIMARY KEY,
                    video_title TEXT NOT NULL,
                    upload_date DATE NOT NULL DEFAULT CURRENT_DATE,
                    upload_status TEXT NOT NULL,
                    privacy_status TEXT NOT NULL,
                    license TEXT NOT NULL,
                    madeForKids BOOLEAN NOT NULL
                    )""")
    curr.execute(create_table_command)


def check_if_video_exists(curr, video_id): 
    query = ("""SELECT video_id FROM VIDEOS WHERE video_id = %s""")
    curr.execute(query, (video_id,)) #video_id as parameter for above command
    return curr.fetchone() is not None #returns single row from table or None

def truncate_table(curr):
    truncate_table = ("""TRUNCATE TABLE videos""")
    curr.execute(truncate_table)


def append_from_df_to_db(curr,df):
    insert_into_videos = ("""INSERT INTO videos (video_id, video_title, upload_date, upload_status, privacy_status, license,madeForKids)
                            VALUES(%s,%s,%s,%s,%s,%s,%s);""")
    for i, row in df.iterrows():
        curr.execute(insert_into_videos, (row['video_id'], row['video_title'], row['upload_date'], row['upload_status'], row['privacy_status'], row['license'], row['madeForKids']))

def update_db(curr,df):
    tmp_df = pd.DataFrame(columns=['video_id', 'video_title', 'upload_date', 'upload_status',
                                   'privacy_status', 'license', 'madeForKids'])
    for i, row in df.iterrows():
        if check_if_video_exists(curr, row['video_id']): # If video already exists then we will update row
            #def update_row(curr, video_id, video_title, upload_status, privacy_status, license, madeForKids):
            #update_row(curr,row['video_id'],row['video_title'],row['upload_status'],row['privacy_status'],row['license'],row['madeForKids'])
            #curr.execute(query, (video_title, upload_status, privacy_status, license, madeForKids, video_id))
            query = ("""UPDATE videos
                        SET video_title = %s,
                            view_count = %s,
                            like_count = %s,
                            dislike_count = %s,
                            comment_count = %s
                        WHERE video_id = %s;""")
            curr.execute(query, (row['video_title'], row['upload_status'], row['privacy_status'], row['license'], row['madeForKids'], row['video_id']))
        else: # The video doesn't exists so we will add it to a temp df and append it using append_from_df_to_db
            tmp_df = tmp_df.append(row)
    return tmp_df

In [26]:
conn = None
conn = connect_to_db('****.rds.amazonaws.com', '***', '***', '***' , '***')

Connected!


In [19]:
curr = conn.cursor()
create_table(curr)
conn.commit()

In [27]:
new_vid_df = update_db(curr,df)
conn.commit()

In [31]:
append_from_df_to_db(curr, new_vid_df)
conn.commit()

In [41]:
import pprint

In [43]:
curr.execute("SELECT * FROM VIDEOS")
pprint.PrettyPrinter().pprint(curr.fetchall())

[('ToEl_tpyoM4',
  'Uber Data Science Python Interview Question Walkthrough',
  datetime.date(2022, 1, 10),
  'processed',
  'public',
  'youtube',
  False),
 ('30hS-MjpU6E',
  'SQL Syntax Best Practices: How to Structure Your SQL Code',
  datetime.date(2021, 12, 22),
  'processed',
  'public',
  'youtube',
  False),
 ('Bgpp99iz0I0',
  'Most Common Coding Mistakes on Data Science Interviews',
  datetime.date(2021, 11, 4),
  'processed',
  'public',
  'youtube',
  False),
 ('GeJUvdkJKEc',
  'Multiple Solutions to Data Scientist Interview Question From Amazon '
  '[Rolling Average]',
  datetime.date(2021, 10, 11),
  'processed',
  'public',
  'youtube',
  False),
 ('GGURenNfXI0',
  'Multiple Approaches for Solving a Twitter Data Scientist Interview Question',
  datetime.date(2021, 9, 13),
  'processed',
  'public',
  'youtube',
  False),
 ('PlpUo6bHsBQ',
  'Advanced Facebook Data Science SQL interview question [RANK()]',
  datetime.date(2021, 8, 9),
  'processed',
  'public',
  'youtube'